# Baseline model: CBOW + MLP classifier

We build here our first baseline model which is the following :

Load embeddings (already processed):
- individual words embedded using word2vec pretrained embeddings 
- CBOW was used to get query / paragraphs vector representations out of the words vectors (try with tf-idf weighted average)

Compare:
- similarity learning (bilinar): learn matrix W through computation of query.W.paragraphT

Classify :
- use final softmax to output probability prediction

Code used inspired from:
- http://www.erogol.com/duplicate-question-detection-deep-learning/

In [2]:
import sys
import os 
import pandas as pd
import numpy as np
import pickle

# load data
df = pickle.load( open( "../data/fold0_with_wrong_with_CBOW_embeddings", "rb" ) )

UnicodeDecodeError: 'ascii' codec can't decode byte 0x92 in position 4: ordinal not in range(128)

In [170]:
df.head()

,pagename,section,para_id,para_text,rel_label,query,query_CBOW,paragraph_CBOW
530192,queensland air museum,aircraft on display,90fdae09838530d8001220d0c963594c74b3c0c7,As of March 2014.,1,aircraft on display of queensland air museum,"[2.54523002729, 0.987329989672, 1.39032000676,...","[1.04178700969, 0.649680987, 0.633059997112, -..."
77501,background of the occupation of the baltic states,europe becomes unstable,721ee539577c3a2c9e75ff393422edf619bde41b,The Soviet Union remained conciliatory with Ba...,1,europe becomes unstable of background of the o...,"[-0.53776698187, 0.524886905681, -0.5034700036...","[8.48787346703, 9.09795231628, -1.21728977765,..."
198858,dirt track racing 2,game modes,2f5b10ded43062a7f40df8357e4a67d431e7a5dc,Multiplayer allows you to race against up to n...,1,game modes of dirt track racing 2,"[2.04991998896, 0.849495004863, -1.66701780166...","[-0.69690399617, -0.88491801545, -2.5799840046..."
195387,dexter walker,storylines,6d3a3149cfc492f79dff3330124d6ab39a9ddea5,Dex is diagnosed with a brain injury leaving ...,1,storylines of dexter walker,"[-0.72718001157, 0.303549099248, 0.10183998569...","[2.70971485808, 24.6431179714, -21.5583597224,..."
177888,criminal record,united kingdom,f16b0995653bd249a5d71934cae333befa7246fb,Information supplied depends on the level of d...,1,united kingdom of criminal record,"[-0.390038985759, 0.0567329889163, 0.371367026...","[-3.37157545811, 5.95376241195, -9.73813020345..."


## Model

In [3]:
# Prepare data

# shuffle df
df = df.reindex(np.random.permutation(df.index))

# set number of train and test instances
num_train = int(df.shape[0] * 0.88)
num_test = df.shape[0] - num_train                 
print("Number of training pairs: %i"%(num_train))
print("Number of testing pairs: %i"%(num_test))

# init data data arrays dim = num_train x (query, paragr) x embedding length
X_train = np.zeros([num_train, 2, 300])
X_test  = np.zeros([num_test, 2, 300])
Y_train = np.zeros([num_train]) 
Y_test = np.zeros([num_test])

# format data 
b = [a[None,:] for a in list(df['query_CBOW'].values)]
query_feats = np.concatenate(b, axis=0)

b = [a[None,:] for a in list(df['paragraph_CBOW'].values)]
paragraph_feats = np.concatenate(b, axis=0)

# fill data arrays with features
X_train[:,0,:] = query_feats[:num_train]
X_train[:,1,:] = paragraph_feats[:num_train]
Y_train = df[:num_train]['rel_label'].values
            
X_test[:,0,:] = query_feats[num_train:]
X_test[:,1,:] = paragraph_feats[num_train:]
Y_test = df[num_train:]['rel_label'].values

# remove useless variables
del b
del query_feats
del paragraph_feats

Number of training pairs: 624290
Number of testing pairs: 85131


In [ ]:
import tensorflow as tf

In [167]:
x = tf.placeholder(tf.float64, shape=300)
y = tf.placeholder(tf.float64, shape=300)
xx = tf.placeholder(tf.float64, shape=[300,2])
yy = tf.placeholder(tf.float64, shape=[300,2])
xx = tf.concat([x, x], 1)
yy = tf.concat([y, y], 1)
out = tf.reduce_sum(tf.mul(yy, tf.reduce_sum(tf.mul(weights['out'], xx))  ) )
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    q = X_train[0,0,:] 
    p = X_train[0,1,:]
    print(sess.run(out, feed_dict={x: q, y: p}))  

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.

In [138]:
# Parameters
training_epochs = 10
batch_size = 100
display_step = 1

# Network Parameters
n_input = 300
n_classes = 2

# tf Graph input

q = tf.placeholder("float64", [None, n_input, n_classes])
p = tf.placeholder("float64", [None, n_input, n_classes])
y = tf.placeholder("float", [None, n_classes])
'''
q = tf.placeholder(tf.float64, shape = n_input)
p = tf.placeholder(tf.float64, shape = n_input)
y = tf.placeholder(tf.float64, shape = n_classes) 
'''

'\nq = tf.placeholder(tf.float64, shape = n_input)\np = tf.placeholder(tf.float64, shape = n_input)\ny = tf.placeholder(tf.float64, shape = n_classes) \n'

In [139]:
# Create model
def bilinar(q, p, weights):
    # Hidden layer with RELU activation
    out = [tf.reduce_sum(tf.mul(q, tf.reduce_sum(tf.mul(weights['out'], p))  ) )]
    return out

In [140]:
y

<tf.Tensor 'Placeholder_95:0' shape=(?, 2) dtype=float32>

In [142]:
# Store layers weight & bias
weights = {'out': tf.Variable(tf.random_normal([n_input, n_input, n_classes],dtype=tf.float64))}

# Construct model
pred = bilinar(q, p, weights)

# Define loss and optimizer
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=pred, labels=y)
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

ValueError: Dimensions must be equal, but are 2 and 300

In [93]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        total_batch = 5
        
        # Loop over all batches
        for i in range(total_batch):
            
            batch_q = np.transpose(X_train[0,0,:]) #tf.constant(X_train[:100][0])
            batch_p = np.transpose(X_train[0,1,:]) #tf.constant(X_train[:100][1])
            batch_y = np.transpose(Y_train[0]) #tf.constant(Y_train[:100])
            '''
            batch_q = X_train[0,0,:] #tf.constant(X_train[:100][0])
            batch_p = X_train[0,1,:] #tf.constant(X_train[:100][1])
            batch_y = Y_train[0] #tf.constant(Y_train[:100])
            '''
              
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={q: batch_q, p: batch_p, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

ValueError: Cannot feed value of shape (300,) for Tensor u'Placeholder_37:0', which has shape '(?, 300)'

In [28]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Lambda, merge, BatchNormalization, Activation, Input, Merge
from keras import backend as K


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_dim):
    '''
    Base network for feature extraction.
    '''
    input = Input(shape=(input_dim, ))
    dense1 = Dense(128)(input)
    bn1 = BatchNormalization(mode=2)(dense1)
    relu1 = Activation('relu')(bn1)

    dense2 = Dense(128)(relu1)
    bn2 = BatchNormalization(mode=2)(dense2)
    res2 = merge([relu1, bn2], mode='sum')
    relu2 = Activation('relu')(res2)    

    dense3 = Dense(128)(relu2)
    bn3 = BatchNormalization(mode=2)(dense3)
    res3 = Merge(mode='sum')([relu2, bn3])
    relu3 = Activation('relu')(res3)  
    
    feats = merge([relu3, relu2, relu1], mode='concat')
    bn4 = BatchNormalization(mode=2)(feats)

    model = Model(input=input, output=bn4)

    return model


def compute_accuracy(predictions, labels):
    '''
    Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def create_network(input_dim):
    # network definition
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    return model

In [131]:
from sklearn import preprocessing
X_train_norm = np.zeros(X_train.shape)
X_train_norm[:,0,:] = preprocessing.normalize(X_train[:,0,:])
X_train_norm[:,1,:] = preprocessing.normalize(X_train[:,1,:])
X_test_norm = np.zeros(X_test.shape)
X_test_norm[:,0,:] = preprocessing.normalize(X_test[:,0,:])
X_test_norm[:,1,:] = preprocessing.normalize(X_test[:,1,:])

In [138]:
# create model
# from siamese import *
from keras.optimizers import RMSprop, SGD, Adam
net = create_network(300)

# train
#optimizer = SGD(lr=1, momentum=0.8, nesterov=True, decay=0.004)
optimizer = Adam(lr=0.001)
net.compile(loss=contrastive_loss, optimizer=optimizer)

for epoch in range(100):
    net.fit([X_train_norm[:,0,:], X_train_norm[:,1,:]], Y_train, # Normalize X_train ?
          validation_data=([X_test_norm[:,0,:], X_test_norm[:,1,:]], Y_test), # Normalize X_test ?
          batch_size=128, nb_epoch=1, shuffle=True, )
    
    # compute final accuracy on training and test sets
    pred = net.predict_classes([X_test_norm[:,0,:], X_test_norm[:,1,:]], batch_size=128) # Normalize X_test ?
    te_acc = compute_accuracy(pred, Y_test)
    
    pred_train = net.predict_classes([X_train_norm[:,0,:], X_train_norm[:,1,:]], batch_size=128) # Normalize X_test ?
    tr_acc = compute_accuracy(pred_train, Y_train)
    
    print(tr_acc)
    
print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Train on 153 samples, validate on 21 samples
Epoch 1/1
153/153 [==============================] - 3s - loss: 693.7363 - val_loss: 686.9468


AttributeError: 'Model' object has no attribute 'predict_classes'

In [54]:
print(te_acc)
print(type(pred[:][0]))
print(type(pred[:][0]))
print(pred[:][0].shape)
print(type(Y_test))
print(Y_test.shape)

nan
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(1,)
<type 'numpy.ndarray'>
(21,)


In [108]:
Y_test

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [73]:
pred = pred.reshape(Y_test.shape).shape

In [135]:
pred_train

array([[  3.39133286],
       [  4.0400672 ],
       [  6.06718016],
       [  4.46551609],
       [  6.1903801 ],
       [  7.62450457],
       [  5.38022614],
       [  4.95393896],
       [  4.19205475],
       [  4.41008043],
       [  4.86572075],
       [  3.5670836 ],
       [  4.99328041],
       [  3.69709849],
       [  7.10036135],
       [  4.63419914],
       [  9.24658585],
       [  3.99168658],
       [  3.63978744],
       [ 11.5053587 ],
       [  4.38359833],
       [  7.02809   ],
       [  4.70760202],
       [  5.80150318],
       [  5.34516954],
       [  4.79439926],
       [  4.41186476],
       [  3.2565546 ],
       [  3.69785547],
       [  9.01588917],
       [  4.04520607],
       [  3.79550385],
       [ 16.75971603],
       [  4.82048416],
       [  6.22091484],
       [  3.45065451],
       [  8.84228706],
       [  8.85808849],
       [  3.87091208],
       [  3.47586226],
       [  7.68869972],
       [  4.40386724],
       [  6.25794458],
       [  9

In [79]:
pred = pred.reshape(Y_test.shape)

In [ ]:
def compute_accuracy(predictions, labels):
    '''
    Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

In [80]:
compute_accuracy(pred, Y_test)

nan

In [81]:
pred

array([ 21.53193092,  40.35143661,  12.72819614,  17.25185585,
        10.99982071,  11.09136677,  11.42963505,  12.64740086,
        16.93758392,  34.72384262,  21.15718269,  11.68271637,
        10.79737282,  10.14541817,  14.79070282,  26.73973083,
        11.13249588,  13.70429611,  45.7140274 ,  38.70048141,  25.32180977], dtype=float32)

In [101]:
Y_test.shape

(21, 1)

In [99]:
Y_test = Y_test.reshape((21,1))

In [123]:
Y_train[pred_train.ravel() < 0.5].mean()

1.0

In [119]:
Y_train[pred_train.ravel() < 0.5]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [122]:
pred_train.ravel()< 0.5

array([ True,  True, False,  True, False, False, False,  True, False,
        True, False,  True,  True,  True,  True,  True, False,  True,
        True, False, False, False,  True, False,  True, False, False,
       False, False, False,  True,  True, False, False, False,  True,
       False, False,  True,  True, False,  True, False, False, False,
        True, False,  True,  True, False,  True, False,  True, False,
       False, False, False,  True,  True,  True,  True, False, False,
        True, False,  True, False, False,  True, False,  True,  True,
       False,  True, False,  True, False, False,  True, False,  True,
        True, False, False, False,  True, False,  True, False,  True,
        True,  True, False,  True, False,  True, False, False, False,
        True, False, False, False, False, False,  True, False,  True,
        True, False, False, False, False,  True,  True, False, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,

In [104]:
Y_test

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])